In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd


e:\miniconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

torch.cuda.is_available() 
print(torch.cuda.device_count())

0


### 라벨 파이프라인

In [ ]:
# [class],[Pc = confidence score],[x], [y], [w], [h],[distance] = 0~7 c x y w h dist

In [17]:
column_names=['class','truncated','occluded','alpha','x1','y1','x2','y2','dimensions_h','dimensions_w','dimensions_len','location_x','location_y','distance','rotation_y']

#txt파일 단위로 불러옴
temp=pd.read_csv('dataset/sample/label/000002.txt',delimiter=' ',names=column_names)
temp=temp.drop(columns=['truncated','occluded','alpha','dimensions_h','dimensions_w','dimensions_len','location_x','location_y','rotation_y'])
temp
# x y w h 형식으로 bbox 변환 (여기서 x y는 중심좌표,)

# 파일크기에 맞게 변환 416 * 416

#class one hot vector로 변환

#이후 데이터셋에서 리스트로 바꿀때 index 8에 c score 자리 만들기

,class,x1,y1,x2,y2,distance
0,Misc,804.79,167.34,995.43,327.94,8.55
1,Car,657.39,190.13,700.07,223.39,34.38


In [19]:
224/7

32.0

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(
            self, csv_file, img_dir, label_dir, S=13, B=5, C=20, transform=None):
        (...)

    def __getitem__(self, index):
        (...)
        label_matrix = torch.zeros((self.S, self.S, self.C + 5*self.B))
        for box in boxes:
            class_label, x, y, width, height = box.tolist()
            class_label = int(class_label)

            # x, y의 위치 grid cell 파악하기
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i
            width_cell, height_cell = (
                    width * self.S,
                    height * self.S
            )

            if label_matrix[i, j, 20] == 0:
                label_matrix[i, j, 20] = 1
                box_coordinates = torch.tensor(
                    [x_cell, y_cell, width_cell, height_cell]
                )
                label_matrix[i, j, 21:25] = box_coordinates
                label_matrix[i, j, class_label] = 1

        return image, label_matrix